# Project 2 : :)
## Authors:

### Warnings

* Ikke endre uten at de andre vet det
* **Alltid** gå på Kernel -> Restart & Clear Output
* Ikke skriv utenfor cellene og sånn

## Implementation of set 1 - points in 2D

In [ ]:
## IMPORTS

import numpy as np
import matplotlib.pyplot as plt
import loader
import plotting
from spirals import get_data_spiral_2d

In [ ]:
## GLOBAL CONSTS AND DEFINITIONS

sigma = lambda x : np.tanh(x)         # Activation function
eta = lambda x : 0.5*(1+np.tanh(x/2)) # Scalar function     
sech = lambda x : 1/np.cosh(x)
eta_ddx = lambda x :  0.25*sech(x/2)*sech(x/2) # Derivative of eta
sigma_ddx = lambda x: sech(x)*sech(x) # Derivative of sigma


K = 4 # Amount of layers, arbitrary number
d = 2 # Ehh
tau = 0.07 # learning parameter [0.01,0.1]
Y0, C = get_data_spiral_2d() #Default n=200
I = Y0.shape[1]
One = np.full((I,2),1)
K = 4 #debug
W, b = np.random.randn(K,d,d), np.random.randn(K,d,1)     #startverdier
mu = 1 #?
w = np.random.randn(d,1)
h = 0.1

#U_i = [W, b, w, mu]

In [ ]:
## INITIALISATION CELL


#optimert funksjon
def YK(W, b):
    #tom Y-matrise, 3-dimensjonalt numpy-array
    Ym = np.zeros((K+1,d,I))
    Ym[0,:,:] = Y0
    k=0
    while k < K:
        Ym[k+1] = Ym[k] + h*sigma(W[k]@Ym[k] + b[k])
        k+=1
    return Ym



#Finner P_K
def getGradients(K, W,b,w,mu):
    Y_all = YK(W,b)
    lastY = Y_all[K-1,:,:]
    P = np.full(Y_all.shape,0) # init matrix
    P[K] = w@np.transpose((Z(w,mu)-C)*eta_ddx(np.transpose(lastY)@w+mu))  #(7)
    dJdMU = eta_ddx(np.transpose(np.transpose(lastY)@w+mu))@(Z(w,mu)-C)   #(5), kunne også brukt .T for transpose
    dJdw = lastY@((Z(w,mu)-C)*eta_ddx(np.transpose(lastY)@w+mu))              #(6)
    for k in range(K,0,-1):
        P[k-1] = P[k]+h*np.transpose(W[k-1,:,:])@(sigma_ddx(W[k-1,:,:]@Y_all[k-1,:,:]+b[k-1,:,:])*P[k])  #(8)  
        #Endra til @
        #Endra første W[k-a] ??
        # feil med 'Y(k-1,W,b)' ??? alle matriser i lag k-1??
    dJdWk = np.zeros((K,d,d))
    dJdBk = np.zeros((K,d,1))
    
    for k in range(0,K-1):
        dJdWk[k] = h*(P[k+1]*sigma_ddx(W[k,:,:]@Y_all[k,:,:]+b[k,:,:]))@np.transpose(Y_all[k,:,:])  #(9)
        dJdBk[k] = h*(P[k+1]*sigma_ddx(W[k,:,:]@Y_all[k,:,:]+b[k,:,:]))@np.ones((I,1)) #One not needed?                     #(10)
    return dJdWk, dJdBk, dJdw, dJdMU


#Adam descent algorithn
def AdamAlg(uParam, U_ind):
    beta1 = 0.9
    beta2 = 0.999
    alpha = 0.01
    epsilon = 1e-8
    g = getGradients(K, W,b,w,mu)[U_ind]
    if firstRun==True:
        lastM = 0
        lastV = 0
    m = beta1*lastM+(1-beta1)*g
    v = beta2*lastV+(1-beta2)*(g*g)
    mhat = m/(1-beta1**(counter)) # NB må altså :)
    vhat = v/(1-beta2**(counter))
    uParam -= alpha*mhat/(np.sqrt(vhat)+epsilon)
    lastM = m
    lastV = v
    return uParam, lastM, lastV

In [ ]:
#Lager siste Y_k = Y_K
YK_list = YK(W,b)
lastY = YK_list[K-1]

Z = lambda w,mu : eta(np.add(np.transpose(lastY)@w,mu))
#print(":)))")
J = 1/2*np.linalg.norm(Z(w,mu)-C)**2 # Cost function

In [ ]:
# Algorithm
firstRun = True
converged = False
counter = 1

#for adamalg

pltJ = []
for i in range(1,40001):
    pltJ.append(J)
    W[0] = AdamAlg(W, 0)
    b[0] = AdamAlg(b, 1)
    w[0] = AdamAlg(w,2)
    mu[0] = AdamAlg(mu,3)
    firstRun = False
    counter = i 
    #converged = True #Only one iteration when debugging
print("W: ", W)
print("b: ", b)
print("w: ", w)
print("mu: ", mu)

plt.plot(i,J,label="Cost function")
plt.legend()
plt.show()

In [ ]:
## QUESTIONS





# First for-loop in getGradients, is the range correct?
# Parameters for Adam alg?
# What is Y[k]? svart
# How to readData()? svart
# How to check if converged? And when? svart
# Hvordan få posisjon til farge? svart
# Må vi ha forskjellgie b_k? Hva menes med b_{k-1}?
# Hva gjør egentlig Adam Descent Alg.? (Hva er U?)
# Random startverdier for vekter og bias: på hvilket intervall??
# Hvor mange verdier for Bk og Wk må vi lage?
# Er Wk=2x2 og Bk=2x1?